# REDCap

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Processing 
In this notebook, we work on processing the home environment surveys.

In [2]:
import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import seaborn as sns
import matplotlib.dates as mdates

# Data Import and Summary

In [3]:
ee = pd.read_csv("../data/raw/utx000/surveys/EESurvey_E1_labels.csv",parse_dates=["Survey Timestamp"],index_col=0)
ee.dropna(subset=["Survey Timestamp"], axis="rows",inplace=True)
ee.drop([column for column in ee.columns if "gender" in column.lower() or "family" in column.lower()],axis="columns",inplace=True)
ee.replace("Unchecked",0,inplace=True)
ee.replace("Checked",1,inplace=True)
ee.head()

,Event Name,Survey Timestamp,"In the past week, have you traveled to a different city?",What type of building are you currently living in? (choice=Apartment),What type of building are you currently living in? (choice=Duplex),What type of building are you currently living in? (choice=Stand-alone House),What type of building are you currently living in? (choice=Dormitory),What type of building are you currently living in? (choice=Motel/Hotel),What type of building are you currently living in? (choice=Other),Are you living with the same people as last week?,"In the past week, have you opened the windows in your residence?",Why did you open your windows? (choice=To change indoor temperature),Why did you open your windows? (choice=To get fresh air),Why did you open your windows? (choice=To change temperature and get fresh air),Why did you open your windows? (choice=Other),"In the past week, have you tried to isolate yourself in some part of your home?",Why? (choice=To be able to work without interruption),Why? (choice=To have more privacy),Why? (choice=To isolate from a sick member of the household),Why? (choice=To isolate myself because I am not feeling well),Why? (choice=Other),How did you try to isolate yourself? (choice=Stayed in bedroom),How did you try to isolate yourself? (choice=Stayed in another room of the house),How did you try to isolate yourself? (choice=Closed doors between rooms),How did you try to isolate yourself? (choice=Closed windows in room),How did you try to isolate yourself? (choice=Tried to change ventilation in the isolated space),How did you try to isolate yourself? (choice=Minimized contact with other household members),How did you try to isolate yourself? (choice=Other),"Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Stagnant)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Smelly)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Earthy)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Moldy)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Cooking)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Fragrant)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Fresh (well ventilated))","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Obnoxious)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Other)","In the past week, have you changed your air conditioning filter?",Are you currently using a portable air cleaner at home?,Did you use the air cleaner prior to the COVID-19 outbreak?,"In the past week, has your house been uncomfortably hot?",Can you easily control the temperature within your home?,"In the past week, has your home been uncomfortably humid (sticky)?","In the past week, have the hard surfaces in your residence been cleaned?",What hard surfaces were cleaned? (choice=Kitchen Counters),What hard surfaces were cleaned? (choice=Floors),What hard surfaces were cleaned? (choice=Door Knobs),What hard surfaces were cleaned? (choice=Table Tops),What hard surfaces were cleaned? (choice=Sinks),What hard surfaces were cleaned? (choice=Toilets),What hard surfaces were cleaned? (choice=Appliance Knobs and Handles),"In the past week, has your home been vacuumed?","In the past week, have any of the following cleaners been used in your home (Check all that apply )? (choice=Bleach)","In the past week, have any of the following cleaners been used in your home (Check all that apply )? (choice=Ammonia)","In the past week, have any of the following cleaners been used in your home (Check all that apply )? (choice=Pinesol)","In the past week, hav

From the raw data, we can see the following subcategories:
1. Building Type
3. Window Opening
4. Isolation
5. Smell
6. Cleaners Used and Location
7. Allergies
8. COVID

## Building Type 
The building types are separated by checked and unchecked boxes. Here we summarize the building type into one column `building_type`

In [4]:
def get_building_type(row):
    for i, building_type in enumerate(["apartment","duplex","house","dormitory","hotel","other"]):
        if row.iloc[i] == 1:
            return building_type
        

In [5]:
ee_building_type = ee[[column for column in ee.columns if "What type of building" in column]]
ee_building_type["building_type"] = ee_building_type.apply(get_building_type, axis="columns")
ee_building_type.drop([column for column in ee_building_type.columns if "What type of building" in column], axis="columns", inplace=True)
ee_building_type.head()

,building_type
Record ID,
1,house
4,apartment
5,apartment
6,house
9,apartment


## Window

In [6]:
def get_window_use(row):
    d = {"use":[]}
    if row.iloc[0] != "No":
        uses = []
        for i, choice in enumerate(["change_temperature","fresh_air","both","other"]):
            if row.iloc[i+1] == 1:
                uses.append(choice)
                
        d["use"] = uses
        
    return d 

In [7]:
ee_window = ee[[column for column in ee.columns if "windows" in column.lower() and "isolate" not in column.lower()]]
ee_window["window_use"] = ee_window.apply(get_window_use, axis="columns")
ee_window.drop([column for column in ee_window.columns if "windows" in column.lower()],axis="columns",inplace=True)
ee_window.head()

,window_use
Record ID,
1,{'use': ['fresh_air']}
4,{'use': []}
5,{'use': []}
6,{'use': []}
9,{'use': ['fresh_air']}


## Smell

In [8]:
def get_smell(row):
    for i, smell_type in enumerate(["stagnant","smelly","earthy","moldy","cooking","fragrant","well_ventilated","obnoxious","other"]):
        if row.iloc[i] == 1:
            return smell_type

In [9]:
ee_smell = ee[[column for column in ee.columns if "smell" in column.lower()]]
ee_smell["smell_type"] = ee_smell.apply(get_smell,axis="columns")
ee_smell.drop([column for column in ee_smell.columns if "smell" in column.lower() and "type" not in column.lower()],axis="columns",inplace=True)
ee_smell.head()

,smell_type
Record ID,
1,well_ventilated
4,stagnant
5,smelly
6,well_ventilated
9,cooking


## Allergy

In [10]:
def get_allergy(row):
    if row.iloc[0] == "No":
        return 0
    else:
        for i, allergy_intensity in enumerate(["significantly_worse","somewhat_worse","same","somewhat_better","significantly_better"]):
            if row.iloc[i] == 1:
                return allergy_intensity
            
    return 0

In [11]:
ee_allergy = ee[[column for column in ee.columns if "allergy" in column.lower()]]
ee_allergy["allergy_intensity"] = ee_allergy.apply(get_allergy,axis="columns")
ee_allergy.drop([column for column in ee_allergy.columns if "allergy" in column.lower() and "intensity" not in column.lower()],axis="columns",inplace=True)
ee_allergy.head()

,allergy_intensity
Record ID,
1,same
4,somewhat_better
5,0
6,somewhat_better
9,same


## Cleaner Use

In [12]:
def get_cleaner_use(row):
    """
    
    """
    d = {"cleaners":[]}
    for i, cleaner in enumerate(["bleach","ammonia","pinesol","vinegar","alcohol","disinfectant_wipes","soap_and_water","floor_cleaners"]):
        cleaners = []
        if row.iloc[i] == 1:
            cleaners.append(cleaner)
        
    d["cleaners"] = cleaners
        
    return d

In [13]:
ee_cleaner_use = ee[[column for column in ee.columns if "cleaners been used" in column.lower()]]
ee_cleaner_use["cleaner"] = ee_cleaner_use.apply(get_cleaner_use, axis="columns")
ee_cleaner_use.drop([column for column in ee.columns if "cleaners been used" in column.lower()], axis="columns", inplace=True)
ee_cleaner_use.head()

,cleaner
Record ID,
1,{'cleaners': []}
4,{'cleaners': []}
5,{'cleaners': []}
6,{'cleaners': ['floor_cleaners']}
9,{'cleaners': []}


## Cleaner Location

In [14]:
def get_cleaner_location(row):
    """
    
    """
    d = {"bleach":[],"ammonia":[],"pinesol":[],"vinegar":[],"alcohol":[],"disinfectant_wipes":[],"soap_and_water":[],"floor_cleaners":[]}
    for cleaner in d.keys():
        temp = row[[column for column in row.index if cleaner in column.replace(" ","_").lower()]]
        locs = []

        for i in range(len(temp)):
            if temp.iloc[i] == 1:
                locs.append(temp.index[i].split("=")[1][:-1])
                
        d[cleaner] = (locs)
        
    return d

In [17]:
ee_cleaner_locs = ee[[column for column in ee.columns if "in which rooms" in column.lower()]]
ee_cleaner_locs["cleaner_locations"] = ee_cleaner_locs.apply(get_cleaner_location,axis="columns")
ee_cleaner_locs.drop([column for column in ee.columns if "in which rooms" in column.lower()],axis="columns",inplace=True)
ee_cleaner_locs.head()

,cleaner_locations
Record ID,
1,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
4,"{'bleach': ['Kitchen', 'Bathroom'], 'ammonia':..."
5,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
6,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
9,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."


# Combining

In [45]:
data = [ee_building_type,ee_window,ee_smell,ee_allergy,ee_cleaner_use,ee_cleaner_locs]
for df in data:
    print("Observations: ",len(df))
    df.sort_index(inplace=True)
df = pd.concat(data,axis="columns")
df.index.rename("redcap",inplace=True)
df.head()

Observations:  55
Observations:  55
Observations:  55
Observations:  55
Observations:  55
Observations:  55


,building_type,window_use,smell_type,allergy_intensity,cleaner,cleaner_locations
redcap,,,,,,
1,house,{'use': ['fresh_air']},well_ventilated,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
4,apartment,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': ['Kitchen', 'Bathroom'], 'ammonia':..."
5,apartment,{'use': []},smelly,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
6,house,{'use': []},well_ventilated,somewhat_better,{'cleaners': ['floor_cleaners']},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
9,apartment,{'use': ['fresh_air']},cooking,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."


In [53]:
id_crossover = pd.read_excel(f'../data/raw/utx000/admin/id_crossover.xlsx',sheet_name='all')
df_final = df.merge(right=id_crossover,left_index=True,right_on="redcap").drop(["first","last"],axis="columns")
df_final.head()

,building_type,window_use,smell_type,allergy_intensity,cleaner,cleaner_locations,redcap,beiwe,beacon,fitbit
1,house,{'use': ['fresh_air']},well_ventilated,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",1,y1tvkx14,NaN,18
3,apartment,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': ['Kitchen', 'Bathroom'], 'ammonia':...",4,idbkjh8u,30.0,22
4,apartment,{'use': []},smelly,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",5,eyf8oqwl,NaN,23
5,house,{'use': []},well_ventilated,somewhat_better,{'cleaners': ['floor_cleaners']},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",6,789gcb6u,NaN,17
7,apartment,{'use': ['fresh_air']},cooking,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",9,fxw5xupi,NaN,NaN


# Combining with other Dataframes

In [56]:
activity = pd.read_csv("../data/processed/fitbit-daily-ux_s20.csv",index_col=0)

In [59]:
activity.drop(["bmi","fat","weight"],axis="columns",inplace=True)

In [60]:
activity["building_type"] = 

,calories,bmr,steps,distance,sedentary_minutes,lightly_active_minutes,fairly_active_minutes,very_active_minutes,calories_from_activities,food_calories_logged,water_logged,beiwe
timestamp,,,,,,,,,,,,
2020-05-13,2781.0,1876.0,9207,4.396294,1241,70,118,11,1097.0,0.0,0.0,hfttkth7
2020-05-14,3727.0,1876.0,15207,7.261114,614,263,134,23,2234.0,0.0,0.0,hfttkth7
2020-05-15,3909.0,1876.0,14556,8.028501,577,205,57,108,2381.0,0.0,0.0,hfttkth7
2020-05-16,3927.0,1876.0,18453,8.748670,760,176,24,151,2364.0,0.0,0.0,hfttkth7
2020-05-17,4180.0,1876.0,15425,7.973149,605,207,50,131,2652.0,0.0,0.0,hfttkth7
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-05,2048.0,1541.0,3103,1.357559,882,160,6,1,527.0,0.0,0.0,e8js2jdf
2020-09-06,1992.0,1541.0,2551,1.116337,782,154,0,0,483.0,0.0,0.0,e8js2jdf
2020-09-07,2180.0,1541.0,5014,2.193546,768,222,0,0,740.0,0.0,0.0,e8js2jdf


In [63]:
df_final["beiwe"].unique()

array(['y1tvkx14', 'idbkjh8u', 'eyf8oqwl', '789gcb6u', 'fxw5xupi',
       'kyj367pi', 'lkkjddam', '8vpj3b9v', 'rnse61g4', 'wnprz3dq',
       'oxcpr7e3', 't4jphih6', 'xxvnhauv', 'hxj6brwj', 'xdbdrk6e',
       'mv14b8aj', 'uu6knbbx', 'hcpu5myv', 'awa8uces', 'rvhdl2la',
       'ewvz3zm1', 'o6xwrota', 'rj4lxgvp', 'nvtfpaor', 'vr9j5rry',
       '9jtzsuu8', '4i7679py', '5fvmg226', '5bg4j1pp', 'tmexej5v',
       'xlw5ntd5', 'shi1ykro', '15tejjtw', 'e73a1pd5', 'q5y11ytm',
       'vpy1a985', '76w6lc1e', 'axk49ssu', 'olaxadz5', 'i4w8dx6l',
       '745vq78e', '1a9udoc5', 'h8aoiyhv', '9xmhtq74', 'itmylz3g',
       'mm69prai', '7dhu3pn7', '43b1cd5m', 'r11k6uxz', 'qh34m4r9',
       '2xtqkfz1', 'hrqrneay', '34483b37', '9q1adgjk', 'i31pt4b4'],
      dtype=object)

In [66]:
activity.merge(right=df_final,on=["beiwe"],how="left").dropna(subset=["building_type"])

,calories,bmr,steps,distance,sedentary_minutes,lightly_active_minutes,fairly_active_minutes,very_active_minutes,calories_from_activities,food_calories_logged,water_logged,beiwe,building_type,window_use,smell_type,allergy_intensity,cleaner,cleaner_locations,redcap,beacon,fitbit
111,2110.0,1887.0,6131,2.851559,1383,57,0,0,266.0,0.0,0.0,r11k6uxz,house,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",67.0,NaN,53
112,2696.0,1887.0,8010,3.749490,1440,0,0,0,0.0,0.0,0.0,r11k6uxz,house,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",67.0,NaN,53
113,2128.0,1887.0,1889,0.886529,1440,0,0,0,0.0,0.0,0.0,r11k6uxz,house,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",67.0,NaN,53
114,2352.0,1887.0,3025,1.411507,1440,0,0,0,0.0,0.0,0.0,r11k6uxz,house,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",67.0,NaN,53
115,1887.0,1887.0,3389,1.590381,1440,0,0,0,0.0,0.0,0.0,r11k6uxz,house,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",67.0,NaN,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,2477.0,1361.0,9712,3.990657,563,475,0,0,1485.0,0.0,0.0,i4w8dx6l,apartment,{'use': []},cooking,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",54.0,40.0,64
4805,2065.0,1361.0,8300,3.400945,618,310,0,0,944.0,0.0,0.0,i4w8dx6l,apartment,{'use': []},cooking,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",54.0,40.0,64
4806,2009.0,1361.0,6179,2.539090,730,311,0,0,878.0,0.0,0.0,i4w8dx6l,apartment,{'use': []},cooking,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",54.0,40.0,64
4807,1834.0,1361.0,3946,1.621574,733,224,0,0,625.0,0.0,0.0,i4w8dx6l,apartment,{'use': []},cooking,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",54.0,40.0,64
